# init funcs

In [1]:
%run ../udp_thingy.ipynb

In [2]:
%cd ../../util_interfaces/
%run advance_dashboard.ipynb

/home/jovyan/work/research_ongoing/20230110_micropython_stuff/util_interfaces
info
        
        Always be aware that this version of dashboard is instanced when defination.
        Which means that there cannot be two dashboard in the same ipynb kernel.
        
        


Box(children=(Button(description='buttoner', disabled=True, layout=Layout(height='30px', width='auto'), style=…

Box(children=(StateButton(description='loop_timer', layout=Layout(height='30px'), style=ButtonStyle(button_col…

In [3]:
R=Responder(("0.0.0.0",8081))

In [4]:
ESP="192.168.2.139"

In [5]:
ESP="192.168.2.101"

In [6]:
ESP="192.168.2.206"

In [7]:
ESP="192.168.2.122"

In [8]:
ESP="192.168.2.132"

In [9]:
ESP="192.168.2.235"

In [10]:
R.to=[(ESP,8080),]

In [11]:
R.send_exe("R")

In [12]:
def rett(task):
    k=ipywidgets.Textarea(layout=ipywidgets.Layout(width='500px', height='100px'))
    task.start()
    return (k,)

@TaskHandlerButtonAdvanced.add_to_state_button(s, rett)
async def control_loop(task, sleep_interval=0.05, keep_len=10):
    global actions
    obs = task.widgets[0]
    
    @loop(sleep_interval)
    def go():
        obs.value="\n".join([str(i) for i in R.ret_list][::-1])
    
    await asyncio.gather(R.loop_recv(),go())
t=control_loop("control_loop")

In [13]:
R.ret_list

[]

In [14]:
s.display()

Box(children=(Button(description='buttoner', disabled=True, layout=Layout(height='30px', width='auto'), style=…

Box(children=(StateButton(description='loop_timer', layout=Layout(height='30px'), style=ButtonStyle(button_col…

Box(children=(StateButton(description='control_loop', layout=Layout(height='30px'), style=ButtonStyle(button_c…

In [15]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# some funcs

In [ ]:
R.send_exe(f"(esp.drums.cancel(),esp.light.duty(0))")

In [49]:
import numpy as np

In [50]:
np.zeros((2,2,2))

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [51]:
k=[]
for x in ["front","behind"]:
    for y in ["left","right"]:
        for z in ["up","side"]:
            k.append("_".join([x,y,z]))
names=np.reshape(k,(2,2,2))
names

array([[['front_left_up', 'front_left_side'],
        ['front_right_up', 'front_right_side']],

       [['behind_left_up', 'behind_left_side'],
        ['behind_right_up', 'behind_right_side']]], dtype='<U17')

In [367]:
import numpy as np
class Kart:
    def __init__(self,):
        self.info_dict=self.init_info()
        
    def init_info(self,):
        dic={}
        
        # positive for
        # right
        # up(wheel_down)
        
        g=np.array([[[-0.15, 0.],
                    [0., 0.]],

                   [[0., 0.],
                    [0.1, 0]]])
        dic["offset"]=g
        
        dic["mul"] = np.array([[[1., -1.],
                                [-1., -1.]],

                               [[-1., -1.],
                                [1., -1.]]])*0.4
        
        dic["arrange"]=[0,1,2,3,4,5,6,7]
        dic["arrange"]=[2,3,5,4,6,7,0,1]
        dic["arrange"]=[2,3,7,6,4,5,0,1]
        
        return dic
    
    def go(self,li):
        ret=li*self.info_dict["mul"]
        
        ret=np.clip(ret,-0.4,0.2)+self.info_dict["offset"]
        ret=ret.flatten()
        ret=ret[self.info_dict["arrange"]]
        
        ret=ret.tolist()
        return ret
    
    def run_no_wait(self,li,add=None):
        li=self.go(li)
        if add is not None:
            li+=add
        R.send_exe(f"esp.pca_moves({li})")
    
    async def run(self,li):
        self.run_no_wait(li)
        await asyncio.sleep(0.2)
        

In [368]:
K=Kart()
q=np.zeros((2,2,2))

await K.run(q)
K.go(q)

[0.0, 0.0, 0.0, 0.1, 0.0, 0.0, -0.15, 0.0]

In [283]:
for ind,u in np.ndenumerate(q):
    q=np.zeros((2,2,2))
    q[ind]=0.5
    print(names[ind],K.go(q))
    
    await K.run(q)
    input()
    

front_left_up [0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0]


front_left_side [0.0, 0.0, 0.0, 0.1, 0.0, 0.0, -0.15, -0.15]


front_right_up [-0.15, 0.0, 0.0, 0.1, 0.0, 0.0, -0.15, 0.0]


front_right_side [0.0, -0.15, 0.0, 0.1, 0.0, 0.0, -0.15, 0.0]


behind_left_up [0.0, 0.0, 0.0, 0.1, -0.15, 0.0, -0.15, 0.0]


behind_left_side [0.0, 0.0, 0.0, 0.1, 0.0, -0.15, -0.15, 0.0]


behind_right_up [0.0, 0.0, 0.0, 0.25, 0.0, 0.0, -0.15, 0.0]


behind_right_side [0.0, 0.0, -0.15, 0.1, 0.0, 0.0, -0.15, 0.0]


In [114]:
li=[0 for _ in range(12)]
R.send_exe(f"esp.pca_moves({li})")

In [134]:
R.send_exe(f"esp.to=[]")

In [135]:
R.send_exe(f"esp.done")

In [120]:
await asyncio.sleep(1)

In [127]:
for q in range(8):
    await asyncio.sleep(0.5)
    li=[0.2 if i==q else 0 for i in range(12)]
    R.send_exe(f"esp.pca_moves({li})")
    
    await asyncio.sleep(0.5)
    li=[0 for _ in range(12)]
    R.send_exe(f"esp.pca_moves({li})")

In [128]:
await asyncio.sleep(0.1)

In [129]:
    R.send_exe(f"esp.pca.off()")

In [130]:
li=[0,0,0,0,0,0,0,0,0.1,0.1]
R.send_exe(f"esp.pca_moves({li})")

In [131]:
li=[0,0,0,0,0,0,0,0,1,1]
R.send_exe(f"esp.pca_moves({li})")

In [132]:
li=[0,0,0,0,0,0,0,0,0,0]
R.send_exe(f"esp.pca_moves({li})")

In [123]:
s.display()

Box(children=(Button(description='buttoner', disabled=True, layout=Layout(height='30px', width='auto'), style=…

Box(children=(StateButton(description='loop_timer', layout=Layout(height='30px'), style=ButtonStyle(button_col…

Box(children=(StateButton(description='control_loop', layout=Layout(height='30px'), style=ButtonStyle(button_c…

In [126]:
R.send_exe("esp.I.scan()")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

# main

In [269]:
q=np.zeros((2,2,2))
K.run_no_wait(q)
K.go(q)

[0.0, 0.0, 0.0, 0.1, 0.0, 0.0, -0.15, 0.0]

In [370]:
import asyncio

In [369]:
def up_to(n,speed=0):
    q=np.zeros((2,2,2))
    q[:,:,0]=n
    K.run_no_wait(q,[speed,speed])
    return K.go(q)

def side_to(to):
    up_to([to,-to])
    
def front_to(to):
    up_to([[to,to],[-to,-to]])

In [307]:
side_to(0)

In [ ]:
to_have=0.6

In [325]:
for i in np.arange(-to_have,to_have,0.1):
    front_to(i)
    await asyncio.sleep(0.2)   

front_to(0)
await asyncio.sleep(0.2)   


In [326]:
for i in np.arange(-to_have,to_have,0.1):
    side_to(i)
    await asyncio.sleep(0.2)   

side_to(0)
await asyncio.sleep(0.2)   


In [371]:
def round_up(speed=0.1):
    q=np.zeros((2,2,2))
    q[:,:,1]=[[-1,1],[1,-1]]
    q[:,:,0]=1
    K.run_no_wait(q,[speed,speed])
    return K.go(q)

In [372]:
round_up(0)

[-0.4, -0.4, 0.2, 0.30000000000000004, -0.4, -0.4, 0.05000000000000002, 0.2]

In [373]:
idle()

In [198]:
def idle():
    q=np.zeros((2,2,2))
    K.run_no_wait(q,[0,0])

In [203]:
idle()

In [374]:
to_have=1

In [375]:
for i in np.arange(-to_have,to_have,0.1):
    front_to(i)
    await asyncio.sleep(0.2)   
idle()
await asyncio.sleep(0.2)   


In [376]:
for i in np.arange(-to_have,to_have,0.1):
    side_to(i)
    await asyncio.sleep(0.2)   
idle()
await asyncio.sleep(0.2)   


In [377]:
for i in np.arange(-to_have,to_have,0.1):
    up_to(i)
    await asyncio.sleep(0.2)   
idle()
await asyncio.sleep(0.2)   


In [378]:
round_up(0)
await asyncio.sleep(0.2)   

for i in np.arange(-to_have,to_have,0.1):
    round_up(i)
    await asyncio.sleep(0.2)   
idle()
await asyncio.sleep(0.2)   


In [ ]:
R.send_exe(f"(esp.drums.cancel(),esp.light.duty(0))")

In [ ]:
R.send_exe("from reluer import MyRelu;actor=MyRelu()")

In [ ]:
R.send_exe("esp.params=actor.unpack_from")

In [ ]:
R.send_exe("R.decode_list=[['4B', 'rec_s_time'],['185f','params']]")
R.encode_list=[['4B', 'get_s_time'], ['185f', 'params']]

In [ ]:
R.send_exe("esp.pi([0,0,1])")

In [ ]:
R.send_exe("esp.done")

In [ ]:
R.send_exe("esp.act.done()")

In [ ]:
R.send_exe("esp.I=I2C(0,scl=Pin(35),sda=Pin(33),freq=1000000)")

In [ ]:
R.send_exe("esp.I.scan()")

In [ ]:
R.send_exe("esp.I")

In [27]:
s.display()

Box(children=(Button(description='buttoner', disabled=True, layout=Layout(height='30px', width='auto'), style=…

Box(children=(StateButton(description='loop_timer', layout=Layout(height='30px'), style=ButtonStyle(button_col…

Box(children=(StateButton(description='control_loop', layout=Layout(height='30px'), style=ButtonStyle(button_c…

In [ ]:
R.send_exe("inp=np.array([1,1,1])")

In [ ]:
R.send_exe('inp=np.dot(inp,actor.params["linear"]["w"])+actor.params["linear"]["b"]')

In [ ]:
R.send_exe("inp=np.maximum(inp,0)")

In [ ]:
R.send_exe("inp")

In [ ]:
R.send_exe('inp=np.dot(inp,actor.params["linear_1"]["w"])+actor.params["linear_1"]["b"]')

In [ ]:
R.send_exe('actor.params.keys()')

In [ ]:
R.send_exe("inp")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("actor")

In [ ]:
s.display()

In [ ]:
R.send_exe(f"esp.to=[]")

In [ ]:
R.send_exe("from machine import Pin,I2C,ADC;an=Pin(5,Pin.IN)")

In [ ]:
R.send_exe("an")

In [ ]:
R.send_exe("ADC(Pin(5)).read()")

In [ ]:
R.send_exe("esp.as1.OUTS")

In [ ]:
R.send_exe("esp.as1.OUTS=1")

In [ ]:
R.send_exe("get_times(100)")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("esp.env.now_step=200")

In [ ]:
R.send_exe("esp.real([2])")

In [ ]:
R.send_exe("esp.real([1])")

In [ ]:
R.send_exe("esp.real([-2])")

In [ ]:
R.send_exe("esp.real([-1])")

In [ ]:
R.send_exe("esp.real([0])")

In [ ]:
R.send_exe("esp.ta=loop.create_task(esp.real.go_to(100,0))")

In [ ]:
R.send_exe("esp.real.get_angle()")

In [ ]:
R.send_exe("esp.act=loop.create_task(learn_loop())")

In [ ]:
R.send_exe("esp.act.cancel()")

In [ ]:
R.send_exe("esp.real([0])")

In [ ]:
R.send_exe("import sys;sys.exit();_=0")

In [ ]:
R.send_exe("esp.as1")

In [ ]:
R.send_exe("k=esp.as1")

In [ ]:
R.send_exe("k.MD")

In [ ]:
R.send_exe("k.ML")

In [ ]:
R.send_exe("k.MH")

In [ ]:
R.send_exe("k.ANGLE")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("time.ticks_ms()")

In [ ]:
s.display()

In [ ]:
R.send_exe("esp")

In [ ]:
R.send_exe("esp.to=[]")

In [ ]:
R.send_exe("esp.to")

In [ ]:
x=63

In [ ]:
x=16383

In [ ]:
R.send_exe(f"esp.m0.frequency({x})")

In [ ]:
R.send_exe("esp.m0.frequency()")

In [ ]:
R.send_exe("esp.m0.speed(0)")

In [ ]:
R.send_exe("esp.m0.speed(50)")

In [ ]:
R.send_exe("esp.m0.speed(-50)")

In [ ]:
R.send_exe("esp.m0.speed(0)")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("esp.real.state")

In [ ]:
s.display()

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

In [ ]:
R.send_exe("")

In [ ]:
R.dict["rec_s_time"]

In [ ]:
def rett(task):
    task.start()
    return []

@TaskHandlerButtonAdvanced.add_to_state_button(s, rett)
async def show_loop(task, sleep_interval=0.05, keep_len=10):
    # global actions
    # obs = task.widgets[0]
    
    @loop(sleep_interval)
    def go():
        import cv2
        cv2.waitKey(1)
        import numpy as np
        get=np.frombuffer(R.dict["eye"],dtype=np.uint8).reshape((25,25))
        get=cv2.rotate(get, cv2.ROTATE_90_CLOCKWISE)
        cv2.imshow("a",cv2.resize(get,(400,400)))
    
    await asyncio.gather(R.loop_recv(),go())
t_s=show_loop("show_loop")

In [ ]:
s.display()

In [ ]:
R.send_exe(f"esp.to=[]")

In [ ]:
R.send_exe("get=loop.create_task(t())")

In [ ]:
R.send_exe("get.cancel()")

In [ ]:
rawData=b"1234"
rawData=rawData[0:0]

In [ ]:
rawData

In [ ]:
from multiprocessing.managers import BaseManager
class QueueManager(BaseManager): pass


QueueManager.register('qi')

m = QueueManager(address=('', 8881), authkey=b'howdy')
m.connect()

q = m.qi()

In [ ]:
s.display()

In [ ]:
def rett(task):
    k=ipywidgets.Textarea(layout=ipywidgets.Layout(width='500px', height='100px'))
    task.start()
    return (k,)

@TaskHandlerButtonAdvanced.add_to_state_button(s, rett)
async def recv_loop(task, sleep_interval=0.2, keep_len=10):
    global actions
    obs = task.widgets[0]
    
    from queue import Empty
    @try_loop(0.01,skip_errors=(Empty,))
    async def do():
        arr=q.get_nowait()
        obs.value=str(arr)
        actions[0]=arr[3]
        n=-arr[4]
        if n>0:
            actions[1]=n
            actions[2]=0
        else:
            actions[1]=0
            actions[2]=-n
        
    await asyncio.gather(do())
        
t=recv_loop("recv_loop")

In [ ]:
s.display()

In [ ]:
q